# Bases de datos

Las bases de datos nos pueden servir como fuente, como destino de los resultados, o ambas. Vamos a ver cómo leer y escribir datos desde Python.

Para estos ejemplos, utilizaremos una base de datos `sqlite` en local.

In [ ]:
import sqlite3
import pandas as pd

## Conexión

Esto varía dependiendo de la BD a la que te vayas a conectar:

* sqlite: es neceario el módulo `sqlite3`, que vamos a utilizar
* PostgreSQL: necesitas el módulo `psycopg2`
* MySQL: hay varias alternativas, como `pymysql` o `mysqlclient`

In [ ]:
conn = sqlite3.connect('080_bd_temporal.sqlite')

## Escritura

Podemos volcar un dataframe a una tabla de la base de datos con [`DataFrame.to_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html).

In [ ]:
alquiler = pd.read_csv('dat/alquiler-madrid-distritos.csv')
alquiler.head()

In [ ]:
alquiler.to_sql('alquiler', conn)

## Consultas

Para lanzar consultas a la base de datos y obtener el resultado en un dataframe, podemos usar [`pd.read_sql()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html)

In [ ]:
query = '''
select distrito, ano, quarter, precio
from alquiler
where distrito = 'Retiro'
and ano between 2012 and 2014
'''

pd.read_sql(query, conn)

#### Ejercicio

Parametriza la query, de forma que el distrito, el año mínimo y el año máximo sean variables. Invoca la query utilizando el argumento `params` de `read_sql`

Parametrizar es importante para evitar inyección de SQL. Esto es especialmente importante si los datos son introducidos por usuarios. [xkcd](http://imgs.xkcd.com/comics/exploits_of_a_mom.png)

## Carga de resultados

Aunque hemos visto que se puede crear y alimentar una tabla en el momento con `to_sql`, lo más habitual es que nuestra base de datos ya tenga las tablas creadas (con su esquema bien definido, índices, claves foráneas, ...) y que queramos añadir datos.

Vamos a crear una tabla definiendo su esquema, para ver como podríamos añadir los datos de un dataframe sin crearla de cero en `to_sql`.

In [ ]:
c = conn.cursor()
c.execute('''create table alquiler_2
             (distrito text, ano integer, quarter integer, precio real)''')

Comprobamos que está vacía

In [ ]:
pd.read_sql('select * from alquiler_2', conn)

Agregamos los datos a la tabla existente

In [ ]:
alquiler.to_sql('alquiler_2', conn, index=False, if_exists='append')

In [ ]:
pd.read_sql('select * from alquiler_2 limit 5', conn) 

## Cierre de conexión

In [ ]:
conn.close()

#### Ejercicio

Crea una BD de sqlite nueva donde cargues información de alojamientos y barrios de Madrid de AirBnB. La información está disponible [aquí](http://insideairbnb.com/get-the-data.html). Los ficheros a cargar son `listings.csv` y `neighbourhoods.csv`.

Una vez cargados, prueba a hacer algunas consultas que sean de tu interés.